In [28]:
# stp 1 Dependecis instal
#!pip install transformers==4.41.2
# Important to install that transformer version, unles it occur error
#See link: https://github.com/THUDM/CogVLM2/issues/181#issuecomment-2381807778
# !pip install transformers sentencepiece datasets evaluate
# !pip install llmlingua

In [1]:
import torch
import json
import time
import os
from transformers import AutoModel, AutoTokenizer, GenerationConfig
from datasets import load_dataset
from tqdm.notebook import tqdm 
import numpy as np
from sklearn.metrics import f1_score
#from LLMLingua.experiments.llmlingua2.evaluation.metrics import qa_f1_score
from metrics import qa_f1_score, f1_score
if 'model' in globals():
    del model 
torch.cuda.empty_cache()

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

#######################################
print('cuda is',torch.cuda.is_available()) 
#################################################
# 3 model & dataset 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cuda is True


In [2]:
print('cuda is',torch.cuda.is_available()) 

cuda is True


In [3]:
# ChatGLM-6B
model = AutoModel.from_pretrained(
    "THUDM/chatglm2-6b-32k",
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto",
    use_cache=True
).to(device)

model=model.eval()
#print('model config is:',model.generation_config)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b-32k", trust_remote_code=True)#.to(device)

In [5]:
# Monkey-patch the tokenizer's _pad method to handle the padding_side argument
original_pad = tokenizer._pad
def new_pad(self, encoded_inputs, max_length=None, padding_strategy="longest", pad_to_multiple_of=None, return_attention_mask=None, **kwargs):
    return original_pad(encoded_inputs, max_length, padding_strategy, pad_to_multiple_of, return_attention_mask)
tokenizer._pad = new_pad.__get__(tokenizer, type(tokenizer))


In [6]:
# Load the four datasets
datasets = ["multifieldqa_en", "hotpotqa", "triviaqa", "narrativeqa"]

In [7]:
#RANDOM token PRUNE
def random_prune(text, compression_ratio=0.5):

    tokens = tokenizer.tokenize(text)#.to(device)  # e.g., ["The", " quick", " brown"...]
    
    num_keep = int(len(tokens) * (compression_ratio)) # compression ratio means remaining 
    
    # Step 3: Randomly pick tokens (like lottery balls)
    kept_indices = sorted(  # Maintain original order
        np.random.choice(  # Random selection
            len(tokens), 
            num_keep, 
            replace=False  # No duplicates
        )
    )
    
    # Step 4: Rebuild text from kept tokens
    pruned_tokens = [tokens[i] for i in kept_indices]
    return tokenizer.convert_tokens_to_string(pruned_tokens)  # Tokens ➔ text

In [8]:
#Baseline

def no_pruning(context: str) -> str:
    """Returns the full context (no pruning)."""
    return context


In [ ]:
def evaluate(data, compression_func, ratio=None):
    f1_scores = []
    latencies = []
    memory_usages = []
    peak_memory_usages = []

    # Model constraints
    max_context_length = 32768  # ChatGLM2-6B-32k's limit
    max_new_tokens = 256  # Allow generating up to 256 tokens
    max_input_length = max_context_length - max_new_tokens  # 32768 - 256 = 32512

    #for idx in range(20): 
    for idx in range(len(data)):  # Test on all examples
        example = data[idx] 
        context = example["context"]
        question = example["input"]
        answers =example["answers"][0]
        if ratio is not None:
            compressed_context = compression_func(context, ratio)
        else:
            compressed_context = compression_func(context)  #for no.prune & compressr

        tokenized = tokenizer.encode(
            compressed_context, 
            truncation=True, 
            max_length=max_input_length
        )
        truncated_context = tokenizer.decode(tokenized, skip_special_tokens=True)
            
        # prompt = f"Context: {compressed_context}\nQuestion: {question}"
        torch.cuda.reset_peak_memory_stats()
        start_time = time.time()
        #response, _ = model.chat(tokenizer, prompt)
        response, _ = model.chat(
            tokenizer,
            question,
            history=[(truncated_context, "Answer:")],
            max_new_tokens=max_new_tokens  # Critical parameter
        )
        latency = time.time() - start_time
        
        # Measure memory
        memory = torch.cuda.memory_allocated() / (1024 ** 2)  # MB
        
        #Measure peak memory?
        peak_memory = torch.cuda.max_memory_allocated() / (1024 ** 2)
        
        # Calculate F1 score
        f1 = qa_f1_score(prediction=response, ground_truth=answers)
        f1_scores.append(f1)
        latencies.append(latency)
        peak_memory_usages.append(peak_memory)
    
    return {
        "avg_f1": np.mean(f1_scores),
        "avg_latency": np.mean(latencies),
        "avg_peak_memory": np.mean(peak_memory_usages)
    }

In [10]:
# Reusable function to print and save results
def log_result_no_prune(message):
    #print(message)
    with open("results_no_prune.txt", "a") as f:  # 'a' = append mode
        f.write(message + "\n")

def log_result_random_prune(message):
    #print(message)
    with open("results_random_prune.txt", "a") as f:  # 'a' = append mode
        f.write(message + "\n")

In [41]:
pruning_ratios = [0.25, 0.5, 0.75]
# for dataset_name in datasets:
dataset_name = datasets[0]
data = load_dataset("THUDM/LongBench", dataset_name, split="test", cache_dir="custom_cache_dir")
# Randn pruning
for ratio in pruning_ratios:
    random_pruning_results = evaluate(data, random_prune, ratio=ratio)
    
    print(f"\n==={dataset_name}  Random Token Pruning ({int(ratio*100)}%) ===")
    print(f"F1: {random_pruning_results['avg_f1']:.2f}, Latency: {random_pruning_results['avg_latency']:.2f}s, Peak Memory: {random_pruning_results['avg_peak_memory']:.2f} MB")
    log_result_random_prune(f"\n==={dataset_name} Random Token Pruning ({int(ratio*100)}%) ===")
    log_result_random_prune(f"F1: {random_pruning_results['avg_f1']:.2f}, Latency: {random_pruning_results['avg_latency']:.2f}s, Peak Memory: {random_pruning_results['avg_peak_memory']:.2f} MB")


===multifieldqa_en  Random Token Pruning (25%) ===
F1: 0.15, Latency: 1.58s, Peak Memory: 24579.09 MB

===multifieldqa_en  Random Token Pruning (50%) ===
F1: 0.26, Latency: 1.30s, Peak Memory: 24955.47 MB

===multifieldqa_en  Random Token Pruning (75%) ===
F1: 0.33, Latency: 1.81s, Peak Memory: 25336.37 MB


In [ ]:
pruning_ratios = [0.25, 0.5, 0.75]
# for dataset_name in datasets:
dataset_name = datasets[1]
data = load_dataset("THUDM/LongBench", dataset_name, split="test", cache_dir="custom_cache_dir")
# Randn pruning
for ratio in pruning_ratios:
    random_pruning_results = evaluate(data, random_prune, ratio=ratio)
    
    print(f"\n==={dataset_name}  Random Token Pruning ({int(ratio*100)}%) ===")
    print(f"F1: {random_pruning_results['avg_f1']:.2f}, Latency: {random_pruning_results['avg_latency']:.2f}s, Peak Memory: {random_pruning_results['avg_peak_memory']:.2f} MB")
    log_result_random_prune(f"\n==={dataset_name} Random Token Pruning ({int(ratio*100)}%) ===")
    log_result_random_prune(f"F1: {random_pruning_results['avg_f1']:.2f}, Latency: {random_pruning_results['avg_latency']:.2f}s, Peak Memory: {random_pruning_results['avg_peak_memory']:.2f} MB")


===hotpotqa  Random Token Pruning (25%) ===
F1: 0.10, Latency: 1.21s, Peak Memory: 12586.63 MB

===hotpotqa  Random Token Pruning (50%) ===
F1: 0.07, Latency: 1.50s, Peak Memory: 13224.21 MB

===hotpotqa  Random Token Pruning (75%) ===
F1: 0.14, Latency: 2.16s, Peak Memory: 13865.97 MB


In [ ]:
pruning_ratios = [0.25, 0.5, 0.75]
# for dataset_name in datasets:
dataset_name = datasets[2]
data = load_dataset("THUDM/LongBench", dataset_name, split="test", cache_dir="custom_cache_dir")
# Randn pruning
for ratio in pruning_ratios:
    random_pruning_results = evaluate(data, random_prune, ratio=ratio-0.1)
    
    print(f"\n==={dataset_name}  Random Token Pruning ({int(ratio*100)}%) ===")
    print(f"F1: {random_pruning_results['avg_f1']:.2f}, Latency: {random_pruning_results['avg_latency']:.2f}s, Peak Memory: {random_pruning_results['avg_peak_memory']:.2f} MB")
    log_result_random_prune(f"\n==={dataset_name} Random Token Pruning ({int(ratio*100)}%) ===")
    log_result_random_prune(f"F1: {random_pruning_results['avg_f1']:.2f}, Latency: {random_pruning_results['avg_latency']:.2f}s, Peak Memory: {random_pruning_results['avg_peak_memory']:.2f} MB")


===triviaqa  Random Token Pruning (25%) ===
F1: 0.46, Latency: 0.65s, Peak Memory: 12567.37 MB

===triviaqa  Random Token Pruning (50%) ===
F1: 0.50, Latency: 1.13s, Peak Memory: 13059.76 MB

===triviaqa  Random Token Pruning (75%) ===
F1: 0.41, Latency: 1.70s, Peak Memory: 13554.41 MB


In [11]:
pruning_ratios = [0.25, 0.5, 0.75]
# for dataset_name in datasets:
dataset_name = datasets[3]
data = load_dataset("THUDM/LongBench", dataset_name, split="test", cache_dir="custom_cache_dir")
# Randn pruning
for ratio in pruning_ratios:
    random_pruning_results = evaluate(data, random_prune, ratio=ratio-0.1)
    
    print(f"\n==={dataset_name}  Random Token Pruning ({int(ratio*100)}%) ===")
    print(f"F1: {random_pruning_results['avg_f1']:.2f}, Latency: {random_pruning_results['avg_latency']:.2f}s, Peak Memory: {random_pruning_results['avg_peak_memory']:.2f} MB")
    log_result_random_prune(f"\n==={dataset_name} Random Token Pruning ({int(ratio*100)}%) ===")
    log_result_random_prune(f"F1: {random_pruning_results['avg_f1']:.2f}, Latency: {random_pruning_results['avg_latency']:.2f}s, Peak Memory: {random_pruning_results['avg_peak_memory']:.2f} MB")

Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_gene


===narrativeqa  Random Token Pruning (25%) ===
F1: 0.11, Latency: 1.74s, Peak Memory: 12668.78 MB


Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_gene


===narrativeqa  Random Token Pruning (50%) ===
F1: 0.15, Latency: 2.75s, Peak Memory: 13865.31 MB


Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_gene


===narrativeqa  Random Token Pruning (75%) ===
F1: 0.14, Latency: 4.54s, Peak Memory: 14929.30 MB


In [44]:
# for dataset_name in datasets:
dataset_name = datasets[0]
data = load_dataset("THUDM/LongBench", dataset_name, split="test", cache_dir="custom_cache_dir")
# Evaluate no pruning (full context)
full_context_results = evaluate(data, no_pruning)
print(f"\n==={dataset_name}  No Pruning (Full Context) ===")
print(f"F1: {full_context_results['avg_f1']:.2f}, Latency: {full_context_results['avg_latency']:.2f}s,  Peak Memory: {full_context_results['avg_peak_memory']:.2f} MB")
log_result_no_prune(f"\n==={dataset_name} No Pruning (Full Context) ===")
log_result_no_prune(f"F1: {full_context_results['avg_f1']:.2f}, Latency: {full_context_results['avg_latency']:.2f}s, Peak Memory: {full_context_results['avg_peak_memory']:.2f} MB")


===multifieldqa_en  No Pruning (Full Context) ===
F1: 0.53, Latency: 1.88s,  Peak Memory: 25723.81 MB


In [45]:
# for dataset_name in datasets:
dataset_name = datasets[1]
data = load_dataset("THUDM/LongBench", dataset_name, split="test", cache_dir="custom_cache_dir")
# Evaluate no pruning (full context)
full_context_results = evaluate(data, no_pruning)
print(f"\n==={dataset_name}  No Pruning (Full Context) ===")
print(f"F1: {full_context_results['avg_f1']:.2f}, Latency: {full_context_results['avg_latency']:.2f}s,  Peak Memory: {full_context_results['avg_peak_memory']:.2f} MB")
log_result_no_prune(f"\n==={dataset_name} No Pruning (Full Context) ===")
log_result_no_prune(f"F1: {full_context_results['avg_f1']:.2f}, Latency: {full_context_results['avg_latency']:.2f}s, Peak Memory: {full_context_results['avg_peak_memory']:.2f} MB")


===hotpotqa  No Pruning (Full Context) ===
F1: 0.28, Latency: 2.99s,  Peak Memory: 26766.88 MB


In [46]:
# for dataset_name in datasets:
dataset_name = datasets[2]
data = load_dataset("THUDM/LongBench", dataset_name, split="test", cache_dir="custom_cache_dir")
# Evaluate no pruning (full context)
full_context_results = evaluate(data, no_pruning)
print(f"\n==={dataset_name}  No Pruning (Full Context) ===")
print(f"F1: {full_context_results['avg_f1']:.2f}, Latency: {full_context_results['avg_latency']:.2f}s,  Peak Memory: {full_context_results['avg_peak_memory']:.2f} MB")
log_result_no_prune(f"\n==={dataset_name} No Pruning (Full Context) ===")
log_result_no_prune(f"F1: {full_context_results['avg_f1']:.2f}, Latency: {full_context_results['avg_latency']:.2f}s, Peak Memory: {full_context_results['avg_peak_memory']:.2f} MB")


===triviaqa  No Pruning (Full Context) ===
F1: 0.44, Latency: 2.30s,  Peak Memory: 26306.77 MB


In [50]:
# for dataset_name in datasets:
dataset_name = datasets[3]
data = load_dataset("THUDM/LongBench", dataset_name, split="test", cache_dir="custom_cache_dir")
# Evaluate no pruning (full context)
full_context_results = evaluate(data, no_pruning)
print(f"\n==={dataset_name}  No Pruning (Full Context) ===")
print(f"F1: {full_context_results['avg_f1']:.2f}, Latency: {full_context_results['avg_latency']:.2f}s,  Peak Memory: {full_context_results['avg_peak_memory']:.2f} MB")
log_result_no_prune(f"\n==={dataset_name} No Pruning (Full Context) ===")
log_result_no_prune(f"F1: {full_context_results['avg_f1']:.2f}, Latency: {full_context_results['avg_latency']:.2f}s, Peak Memory: {full_context_results['avg_peak_memory']:.2f} MB")

Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_gene


===narrativeqa  No Pruning (Full Context) ===
F1: 0.17, Latency: 6.04s,  Peak Memory: 29049.48 MB
